In [ ]:
from braininversion.BiotSolver import solve_biot
import numpy as np
from fenics import *
from mshr import *
import matplotlib.pyplot as plt
import numpy as np
from fenics_adjoint import *
from braininversion.PlottingHelper import (plot_pressures_and_forces_timeslice, 
                            plot_pressures_and_forces_cross_section,
                            extract_cross_section, style_dict)


T = 1.2            # final time
num_steps = 10     # number of time steps
dt = T / num_steps # time step size
times = np.linspace(0, T, num_steps)


kappa = 15*(1e-9)**2     # permeability 15*(1e-9)**2
visc = 0.8*1e-3          # viscocity 
K = Constant(kappa/visc) # hydraulic conductivity
E = 1500.0 # Young modulus
nu = 0.479 # Poisson ratio

c = 2*1e-4  # storage coefficent
alpha = 1.0 # Biot-Willis coefficient (change of the bulk volume due to pore pressure change)


# Lame coefficients
lmbda = nu*E/((1.0-2.0*nu)*(1.0+nu)) 
mu = E/(2.0*(1.0+nu))

mmHg2Pa = 133.32
material_parameter = dict()
material_parameter["c"] = c
material_parameter["K"] = K 
material_parameter["lmbda"] = lmbda
material_parameter["mu"] = mu
material_parameter["alpha"] = alpha


# create mesh
size = 0.1
N = 15
mesh = RectangleMesh(Point(0.0, 0.0), Point(size, size), N, N)

A = 2*mmHg2Pa
f = 1
p_obs = Expression("A*sin(2*pi*f*t)", A=A, f=f, t=0, degree=2)

ventricle = CompiledSubDomain("on_boundary && near(x[0], 0.0)")
skull = CompiledSubDomain("on_boundary && near(x[0], size)", size=size)

boundary_marker = MeshFunction("size_t", mesh, 1)
ventricle.mark(boundary_marker, 2)
skull.mark(boundary_marker, 1)

n = FacetNormal(mesh)
boundary_conditions_u = {2:{"Neumann":n*p_obs},  #skull
                         1:{"Dirichlet":Constant((0.0, 0.0))}} # ventricle

boundary_conditions_p = {1:{"Dirichlet":p_obs},
                         2:{"Dirichlet":p_obs}}

f = Constant((0.0,0.0))
g = Constant(0.0)

y = 0.5*size
x_coords = np.linspace(0.0, size, 1000)
slice_points = [Point(x,y) for x in x_coords]

In [ ]:
solution = solve_biot(mesh, f, g, T, num_steps, material_parameter,
                      boundary_marker, boundary_conditions_p,
                      boundary_marker, boundary_conditions_u,
                     solver_type="krylov")
solution = [s.copy() for s in solution]

In [ ]:
for s in solution:
    u, pT, p = s.split()
    plt.subplots(1,3, figsize=(15,7))
    plt.subplot(1,3,1)
    plot(u)
    plt.subplot(1,3,2)
    c = plot(pT)
    plt.colorbar(c)

    plt.title("total pressure")
    plt.subplot(1,3,3)
    c = plot(p)
    plt.colorbar(c)
    plt.title("fluid pressure")

In [ ]:
fluid_pressure = [s.split()[2] for s in solution]
disp = [s.split()[0] for s in solution]
total_pressure = [s.split()[1] for s in solution]

pressures = {"p_fluid" : extract_cross_section(fluid_pressure, slice_points)/mmHg2Pa,
             "p_total" : -1*extract_cross_section(total_pressure, slice_points)/mmHg2Pa,
            }
#cdpdt = np.diff(pressures["p_opt_dirichlet"],n=1, axis=0, prepend=0)/dt*c

forces = {"disp_x" : extract_cross_section(disp, slice_points)
         }
for i in range(num_steps): 
    plot_pressures_and_forces_cross_section(pressures, forces, i, x_coords)
    plt.suptitle(f"t = {times[i]:.3f}")
    plt.ylabel("f in N")


In [ ]:
class OneExpression(Expression):
    def eval(self, value, x):
        value[0] = 1.0
    def value_shape(self):
        return (1,)

def rigid_motions(mesh):

    gdim = mesh.geometry().dim()
    x = SpatialCoordinate(mesh)
    c = np.array([assemble(xi*dx) for xi in x])
    volume = assemble(Constant(1)*dx(domain=mesh))
    c /= volume
    c_ = c
    c = Constant(c)

    if gdim == 1:       
        translations = [(OneExpression())]        
        return translations
    
    if gdim == 2:
        translations = [Constant((1./sqrt(volume), 0)),
                        Constant((0, 1./sqrt(volume)))]

        # The rotational energy
        r = assemble(inner(x-c, x-c)*dx)

        C0, C1 = c.values()
        rotations = [Expression(('-(x[1]-C1)/A', '(x[0]-C0)/A'), 
                                C0=C0, C1=C1, A=sqrt(r), degree=1)]
        
        return translations + rotations

    if gdim == 3:
        # Gram matrix of rotations
        R = np.zeros((3, 3))

        ei_vectors = [Constant((1, 0, 0)), Constant((0, 1, 0)), Constant((0, 0,1))]
        for i, ei in enumerate(ei_vectors):
            R[i, i] = assemble(inner(cross(x-c, ei), cross(x-c, ei))*dx)
            for j, ej in enumerate(ei_vectors[i+1:], i+1):
                R[i, j] = assemble(inner(cross(x-c, ei), cross(x-c, ej))*dx)
                R[j, i] = R[i, j]

        # Eigenpairs
        eigw, eigv = np.linalg.eigh(R)
        if np.min(eigw) < 1E-8: warning('Small eigenvalues %g' % np.min(eigw))
        eigv = eigv.T
        # info('Eigs %r' % eigw)

        # Translations: ON basis of translation in direction of rot. axis
        # The axis of eigenvectors is ON but dont forget the volume
        translations = [Constant(v/sqrt(volume)) for v in eigv]

        # Rotations using the eigenpairs
        C0, C1, C2 = c_

        def rot_axis_v(pair):
            '''cross((x-c), v)/sqrt(w) as an expression'''
            v, w = pair
            return Expression(('((x[1]-C1)*v2-(x[2]-C2)*v1)/A',
                               '((x[2]-C2)*v0-(x[0]-C0)*v2)/A',
                               '((x[0]-C0)*v1-(x[1]-C1)*v0)/A'),
                               C0=C0, C1=C1, C2=C2, 
                               v0=v[0], v1=v[1], v2=v[2], A=sqrt(w),
                               degree=1)
        # Roations are discrebed as rot around v-axis centered in center of
        # gravity 
        rotations = list(map(rot_axis_v, zip(eigv, eigw)))
   
        return translations + rotations


In [ ]:
rm = rigid_motions(mesh)


In [ ]:
rm_func = [project(rmi , VectorFunctionSpace(mesh, "CG", 1)) for rmi in rm]

In [ ]:
null_space = VectorSpaceBasis([rm.vector() for rm in rm_func])


In [ ]:
null_space

In [ ]:
rm